# Into Machine Learning

## 101 Training & Testing & Cross Validation

数据集的处理至关重要，独立的测试集可以给算法进行评估，并且避免过度拟合的情况。

### Features and Labels(特征和标签)

特征是输入值，标签是输出值，我想要得到的算法就是可以输入数据特征，预测其可能的标签

Features are individual measurable properties that you will be using to make predictions about your labels.

### Train/Test split in sklearn

首先，我学习如何将数据集分割成训练集和测试集。从库中自带的 datasets 中取一组数据作为研究

In [13]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm

iris = datasets.load_iris()
# df = pd.DataFrame(iris)
print iris.data[:5]
print iris.target[:5]

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]]
[0 0 0 0 0]


这组数据的输入部分为 data 输出部分为 target

每一个数据拥有四个特征，和一个标签，目的就是通过这四个特征，来预测其标签。

接下来采用 cross_validation 中的 train_test_split 来分割数据集。 train_test_split 需要的内容有
- 原始数据集的输入部分
- 原始数据集的输出部分
- 测试集占原始数据集的比例
- 一个随机数种子

In [29]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(iris.data, iris.target, test_size=0.4, random_state=4)
x_train.shape, x_test.shape

((90L, 4L), (60L, 4L))

接下来我创建一个 SVM 的分类器对训练集进行分析并将得到的方法运用于测试集进行算法评价。

In [38]:
clf = svm.SVC(kernel='linear', C=1).fit(x_train,y_train)

通过训练得到的 clf， 我可以对新的数据进行预测，比如：

# 

In [47]:
clf.predict(np.array([ 5.1,3.5,2,0.2]))

C:\ProgramData\Anaconda3\envs\udacity\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([0])

现在，可以通过测试集对 clf 的预测结果进行评价

In [49]:
clf.score(x_test, y_test)

0.98333333333333328

### cross validation(交叉验证)

虽然我们分割了训练集和测试集，但是仍然存在过拟合的风险，所以采用cross validation(交叉验证)来避免风险。

最简单的方法就是使用 cross_val_score 进行评估

In [55]:
clf = svm.SVC(kernel='linear',C=1) #创建一个空的分类器

# 基于原始数据集对分类器进行五次交叉验证
scores = cross_validation.cross_val_score(clf, iris.data, iris.target, cv=5)
scores

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

得到五次交叉验证的准确率，每次验证的评估方法默认采用 score method ，也可以进行指定，比如 f1_weighted method

In [57]:
cross_validation.cross_val_score(clf, iris.data, iris.target, cv=5, scoring='f1_weighted')

array([ 0.96658312,  1.        ,  0.96658312,  0.96658312,  1.        ])

cross_val_score 中的 cv 为整数时，默认采用k-fold 或者 stratifiedkfold 进行交叉验证，我也可以自己定义交叉验证的划分方法

In [60]:
n_samples = iris.data.shape[0]
cv = cross_validation.ShuffleSplit(n_samples, n_iter=3, test_size=0.3, random_state=0)
cross_validation.cross_val_score(clf, iris.data, iris.target, cv=cv)

array([ 0.97777778,  0.97777778,  1.        ])

### GridSearchCV

为了让算法更加真实和精确，只对数据集进行交叉验证是不够的，对于一个算法，其中的参数也要进行不断的调整组合进行尝试。以SVM算法为例

In [ ]:
from sklearn import grid_search
parameters = {'kernel':('linear','rbf'),'C':[1,10]} # 指定变化的参数，这里是两种核函数和两个C值
svc = svm.SVC() # 指定向量机算法
clf = grid_search.GridSearchCV(svc, parameters) # 对上一步的算法中参数进行网格搜索
clf.fit(iris.data,iris.target) # 采用网格搜索中所有参数组合的算法对数据进行拟合